# 自定义 Remote Tool

当前企业中大部分的功能都是通过 API 的形式暴露，Agent如果想要拓展自己的能力边界，就必须基于现有的功能性 API（eg：查天气或查火车票的 api）来进行交互，从而实现更复杂的企业级功能。

Agent想与存量功能性 API 进行交互需要有一个标准的交互协议，而 ERNIE-Bot-Agent 中已经提供了 RemoteTool 和 RemoteToolkit 来简化此交互流程，接下来将介绍 如何在 ERNIE-Bot-Agent 中使用 RemoteTool。

## 使用 RemoteTool

RemoteTool（远程工具）可以是开发者自己提供的，也可以上 AI Studio 的工具中心搜索对应工具，使用代码如下所示：

### 开发者提供的RemoteTool

在开始本教程前，我们需要先获取[飞桨AI Studio星河社区的access_token](https://aistudio.baidu.com/index/accessToken)并且其配置成环境变量，用于对调用大模型和工具中心进行鉴权。

In [1]:
## 添加环境环境变量

import os
os.environ["EB_AGENT_ACCESS_TOKEN"] = "<access_token>"

os.environ["EB_AGENT_LOGGING_LEVEL"] = "info"

from IPython import get_ipython
get_ipython().system = os.system

In [ ]:
!pip install -r requirements.txt
!python server.py &

0

 * Serving Flask app 'server'
 * Debug mode: on


 * Running on http://127.0.0.1:8081
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
 * Debugger is active!
 * Debugger PIN: 239-678-979


In [3]:
from erniebot_agent.tools.remote_toolkit import RemoteToolkit
from erniebot_agent.agents.function_agent import FunctionAgent
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import WholeMemory

In [4]:
toolkit = RemoteToolkit.from_url("http://127.0.0.1:8081")  # 必须存在：http://xxx.com/.well-known/openapi.yaml
llm = ERNIEBot("ernie-3.5")
agent = FunctionAgent(llm, tools=toolkit.get_tools(), memory=WholeMemory())
result = await agent.run("添加一个单词“red”到我的单词本")
print(result.text)

127.0.0.1 - - [29/Dec/2023 17:53:01] "GET /.well-known/openapi.yaml HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:01] "HEAD /.well-known/examples.yaml HTTP/1.1" 404 -
INFO - [Run][Start] FunctionAgent is about to start running with input:
添加一个单词“red”到我的单词本
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: 添加一个单词“red”到我的单词本 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "单词本/v1/addWord",
  "thoughts": "用户想要添加一个单词到单词本，我可以调用单词本/v1/addWord工具来实现这个需求。",
  "arguments": "{\"word\":\"red\"}"
} 
INFO - [Tool][Start] RemoteTool is about to start running with input:
{
  "word": "red"
}
127.0.0.1 - - [29/Dec/2023 17:53:04] "POST /add_word?version=v1 HTTP/1.1" 200 -
INFO - [Tool][End] RemoteTool finished running with output:
{
  "message": "单词添加成功",
  "prompt": "请避免使用\"根据提供的内容、文章、检索结果……\"等表述，不要做过多的解释。"
}
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: function 
 name: 单词本/v1

当然可以！请注意，这将在现有的单词本中添加一个新单词。

我已经为您的单词本添加了单词“red”。


In [5]:
result = await agent.run("单词本当中有哪些单词呢？")
print(result.text)

INFO - [Run][Start] FunctionAgent is about to start running with input:
单词本当中有哪些单词呢？
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: 单词本当中有哪些单词呢？ 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "单词本/v1/getWordbook",
  "thoughts": "用户想要查看他的单词本中的单词。",
  "arguments": "{}"
} 
INFO - [Tool][Start] RemoteTool is about to start running with input:
{}
127.0.0.1 - - [29/Dec/2023 17:53:10] "GET /get_wordbook?version=v1 HTTP/1.1" 200 -
INFO - [Tool][End] RemoteTool finished running with output:
{
  "prompt": "请避免使用\"根据提供的内容、文章、检索结果……\"等表述，不要做过多的解释。",
  "wordbook": [
    "red"
  ]
}
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: function 
 name: 单词本/v1/getWordbook 
 content: {"prompt": "请避免使用\"根据提供的内容、文章、检索结果……\"等表述，不要做过多的解释。", "wordbook": ["red"]} 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 content: 您的单词本中现在有以下单词：

1. red

如果您想添加更多单词，请随时告诉我

您的单词本中现在有以下单词：

1. red

如果您想添加更多单词，请随时告诉我！


以上展示了如何启动一个本地 RemoteTool 并在 ERNIE-Bot-Agent 中调用，使用步骤和 LocalTool 一样。

开发者可基于 [./erniebot_agent/examples/remote-tool](../examples/remote-tools/README.md) 目录下的代码进行二次开发，实现自定义模块的业务 Tool。

本地 RemoteTool Server 主要包含两部分：

1. 本地 restful api 的服务。
2. openapi.yaml 描述文件，主要是为了提供 API 的元信息

### 使用 AI Studio 远程工具

AI Studio 工具中心包含大量稳定服务，开发者可直接调用其工具实现自定义功能，比如以下调用百度翻译的远程工具，

In [6]:
toolkit = RemoteToolkit.from_aistudio("text-moderation")
agent = FunctionAgent(llm=ERNIEBot(model="ernie-3.5"), tools=toolkit.get_tools())
result = await agent.run("“我明天出去玩”这句话合规吗？")
print(result.text)


INFO - [Run][Start] FunctionAgent is about to start running with input:
“我明天出去玩”这句话合规吗？
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: user 
 content: “我明天出去玩”这句话合规吗？ 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 function_call: 
{
  "name": "text-moderation/v1.2/text_moderation",
  "thoughts": "用户想要知道“我明天出去玩”这句话是否合规。这需要审核文本的合规性。",
  "arguments": "{\"text\":\"我明天出去玩\"}"
} 
INFO - [Tool][Start] RemoteTool is about to start running with input:
{
  "text": "我明天出去玩"
}
INFO - [Tool][End] RemoteTool finished running with output:
{
  "conclusion": "合规",
  "isHitMd5": false,
  "conclusionType": 1
}
INFO - [LLM][Start] ERNIEBot is about to start running with input:
 role: function 
 name: text-moderation/v1.2/text_moderation 
 content: {"conclusion": "合规", "isHitMd5": false, "conclusionType": 1} 
INFO - [LLM][End] ERNIEBot finished running with output:
 role: assistant 
 content: 这句话在语法和表达上并没有问题，是一个简单的主谓宾句。主语是“我”，谓语是“出去玩”，时间状语是“明天”。

但是，是

这句话在语法和表达上并没有问题，是一个简单的主谓宾句。主语是“我”，谓语是“出去玩”，时间状语是“明天”。

但是，是否合规还需要考虑具体的语境和背景。例如，如果是在工作时间或学校上课时间说这句话，可能就不太合适。此外，也要看是否有出行的相关计划和准备，如果只是随便说说而没有实际安排，可能会让人感到不负责任。

所以，从语法上看，“我明天出去玩”这句话是合规的，但在实际使用时，还需要考虑具体的语境和背景。


## RemoteTool vs RemoteToolkit

RemoteTool 是单个远程工具，比如添加单词到单词本功能属于单个 RemoteTool，可是：添加单词、删除单词和查询单词这几个功能组装在一起就组成了一个 Toolkit（工具箱），故称为 RemoteToolkit。

以下将会统一使用 RemoteTool 来标识远程工具。

## RemoteTool 如何与 Agent 交互

无论是 LocalTool 还是 RemoteTool 都必须要提供核心的信息：

* tool 的描述信息
* tool 的输入和输出参数
* tool 的执行示例

LocalTool 是通过代码定义上述信息，而 RemoteTool 则是通过`openapi.yaml`来定义上述信息，RemoteToolkit 在加载时将会解析`openapi.yaml`中的信息，并在执行时将对应 Tool 的元信息传入 Agent LLM 当中。

此外 RemoteTool 的远端调用是通过 http 的方式执行，同时遵照 [OpenAPI 3.0](https://swagger.io/specification/) 的规范发送请求并解析响应。OpenAPI.yaml 文件示例如下所示：

* openapi.yaml

```yaml
openapi: 3.0.1
info:
    title: 单词本
    description: 个性化的英文单词本，可以增加、删除和浏览单词本中的单词，背单词时从已有单词本中随机抽取单词生成句子或者段落。
    version: "v1"
servers:
    - url: http://127.0.0.1:8081
paths:
    /add_word:
        post:
            operationId: addWord
            description: 在单词本中添加一个单词
            requestBody:
                required: true
                content:
                    application/json:
                        schema:
                            $ref: "#/components/schemas/addWord"
            responses:
                "200":
                    description: 单词添加成功
                    content:
                        application/json:
                            schema:
                                $ref: "#/components/schemas/messageResponse"

components:
    schemas:
        addWord:
            type: object
            required: [word]
            properties:
                word:
                    type: string
                    description: 需要添加到单词本中的一个单词
        messageResponse:
            type: object
            required: [message]
            properties:
                result:
                    type: string
                    description: 回复信息
```